In [28]:
import pandas as pd
import geopandas as gpd
from modules import preprocess

pd.set_option('display.max_columns', 500)
state_selector = ['08', '20', '31', '40', '46', '48'] 
static_features = ["awc", "bulk_density", "drainage_class_1", "drainage_class_2", "drainage_class_3", "drainage_class_4", "drainage_class_5", "drainage_class_6"]
temporal_prefixes = ["tavg", "prec", "tmin", "tmax", "ndvi", "fpar", "rad", "et0", "cwb", "ssm", "rsm"]
# Enable autoreload for Jupyter notebooks
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1. Define study crop and country

In [43]:
# USER INPUTS
country = "US" # one of ["US", "BR"]
crop = "wheat" # one of ["maize", "wheat"]

shapefile_path, crop_season_in_days_of_year, crop_season_in_months, offset, test_years = preprocess.get_study_metadata(country, crop)
crop_season_in_days_of_year = tuple([1, 365])
crop_season_in_months = tuple([8, 7])
print(shapefile_path, crop, country, crop_season_in_days_of_year, crop_season_in_months, offset, test_years)

../data/shapefiles/US/tl_2023_us_county.shp wheat US (1, 365) (8, 7) 19 [2021, 2022, 2023]


### 2. Read data

From CY-Bench, we have five predictor datasets. 
 
| **ID** | **Name**        | **Time** | **Variables**                         | **Steps**                                    | 
|--------|-----------------|----------|---------------------------------------|----------------------------------------------|
| 1      | FPAR            | bins     | fpar                                  | filter adm_ids, filter crop season           |            
| 2      | METEO           | daily    | tmin, tmax, prec, rad, tavg, et0, cwb | filter adm_ids, filter crop season,<br>resample to 8-day bins  |        
| 3      | NDVI            | bins     | ndvi                                  | filter adm_ids, filter crop season, pivotieren           |           
| 4      | SOIL MOISTURE   | daily    | ssm, rsm                              | filter adm_ids, filter crop season, <br>resample to 8-day bins, pivotieren  |          
| 5      | SOIL            | static   | awc, bulk_density, drainage_class     | filter adm_ids, pivotieren                                |         


In [30]:
# CY-Bench data
fpar = pd.read_csv("../data/CY-Bench/{}/{}/fpar_{}_{}.csv".format(country, crop, crop, country)) 
meteo = pd.read_csv("../data/CY-Bench/{}/{}/meteo_{}_{}.csv".format(country, crop, crop, country), usecols=['crop_name','adm_id','date','rad','et0','cwb', 'tavg', 'tmin', 'tmax', 'prec'])
ndvi = pd.read_csv("../data/CY-Bench/{}/{}/ndvi_{}_{}.csv".format(country, crop, crop, country)) 
soil_moisture = pd.read_csv("../data/CY-Bench/{}/{}/soil_moisture_{}_{}.csv".format(country, crop,  crop, country))
soil = pd.read_csv("../data/CY-Bench/{}/{}/soil_{}_{}.csv".format(country, crop, crop, country)) 
yield_data = pd.read_csv("../data/CY-Bench/{}/{}/yield_{}_{}.csv".format(country, crop, crop, country))

### 3. Preprocess

In [31]:
cy_bench_data = [fpar, soil_moisture, ndvi, soil, meteo]
relevant_adm_ids = yield_data["adm_id"].unique()
cy_bench_data = preprocess.filter_predictors_by_adm_ids(cy_bench_data, relevant_adm_ids)
fpar, soil_moisture, ndvi, soil, meteo = cy_bench_data

In [52]:
temporal_cy_bench_data = [fpar, soil_moisture, ndvi, meteo]
temporal_cy_bench_data = preprocess.preprocess_temporal_data(temporal_cy_bench_data, crop_season_in_months, crop_season_in_days_of_year, crop, country)
 
fpar, soil_moisture, ndvi, meteo = temporal_cy_bench_data

In [58]:
# merge
predictors = (soil_moisture
              .merge(ndvi 
                     .merge(meteo
                            .merge(fpar, on=["adm_id", "harvest_year"], how="inner"), 
                            on=["adm_id", "harvest_year"], how="inner"), 
                     on=["adm_id", "harvest_year"], how="inner")
              .merge(soil.drop("crop_name", axis=1), on="adm_id", how="left"))

# one-hot encode drainage_class and drop
predictors = (predictors
              .join(pd.get_dummies(predictors["drainage_class"].astype('Int64'), dtype=int, prefix="drainage_class"))
              .drop("drainage_class", axis=1))

In [59]:
predictors = preprocess.interpolate_fpar_timesteps(predictors, "tavg")
predictors = preprocess.order_temporal_features(predictors, temporal_prefixes)
predictors = predictors.assign(state=predictors["adm_id"].apply(lambda x: x.split("-")[1]))
predictors = predictors.loc[predictors["state"].isin(['08', '20', '31', '40', '46', '48'])].reset_index(drop=True)
predictors = predictors.drop("state", axis=1)

In [61]:
predictors_16day_bins = preprocess.temporal_aggregation_from8day_to_window(predictors, temporal_prefixes, 2)

Now we have two predictor datasets:

- predictors: where the temporal features are in 8day bins
- predictors_16day_bins: where the temporal features are in 16day bins

### Add additional features

### lat, lon of adm_id


In [62]:
us_shape = gpd.read_file(shapefile_path)

us_shape = us_shape.assign(adm_id="US-"+us_shape["STATEFP"]+"-"+us_shape["COUNTYFP"],
                centroid=us_shape["geometry"].to_crs(epsg=32723).centroid)[["adm_id", "geometry", "centroid"]]

county_centroids = us_shape.assign(lon=us_shape["centroid"].to_crs(epsg=4326).x, lat=us_shape["centroid"].to_crs(epsg=4326).y).drop(["centroid", "geometry"], axis=1)

predictors = predictors.merge(county_centroids, on="adm_id", how="left")
predictors_16day_bins = predictors_16day_bins.merge(county_centroids, on="adm_id", how="left")

### yield of past years

In [65]:
yield_data = yield_data.loc[(yield_data["adm_id"].isin(predictors["adm_id"].unique())) & (yield_data["harvest_year"].between(1999, 2023))].reset_index(drop=True)

yield_data_pivot = yield_data.pivot(index="adm_id", columns="harvest_year", values="yield")
yield_data_pivot = (pd.DataFrame(index=pd.MultiIndex.from_frame(yield_data[["adm_id", "harvest_year"]]))
                  .reset_index(level=1).rename(columns={"level_1": "harvest_year"})
                  .merge(yield_data_pivot, left_index=True, right_index=True, how="left")
                  .reset_index(drop=False))
yield_data_pivot = yield_data_pivot.loc[yield_data_pivot["harvest_year"].between(2004, 2023)].reset_index(drop=True)
yield_data_pivot = yield_data_pivot.set_index(["adm_id", "harvest_year"])

In [66]:
li = []
for row in yield_data_pivot.index:
    temp = yield_data_pivot.loc[row, list(range(row[1]-5, row[1]))]
    temp = pd.DataFrame(dict(zip([f"yield_{i}" for i in range(-5, 0)], temp.values), index=[0])).drop(columns=["index"])
    temp = temp.interpolate(axis=1, limit_direction="both")
    temp = temp.set_index(pd.MultiIndex.from_tuples([row])).reset_index().rename(columns={"level_0": "adm_id", "level_1": "harvest_year"})
    li.append(temp)

yield_past_years = pd.concat(li, axis=0)
yield_past_years = yield_past_years.dropna().reset_index(drop=True)

In [67]:
yield_past_years.head(2)

,adm_id,harvest_year,yield_-5,yield_-4,yield_-3,yield_-2,yield_-1
0,US-08-001,2010,2.2882,2.2882,2.2882,1.91805,2.45645
1,US-08-001,2009,2.2882,2.2882,2.2882,2.28820,1.91805


#### Merge with predictors

In [68]:
predictores_with_yield = predictors.merge(yield_past_years, on=["adm_id", "harvest_year"], how="inner")
predictores_16day_bins_with_yield = predictors_16day_bins.merge(yield_past_years, on=["adm_id", "harvest_year"], how="inner")

# EXPORT

In [69]:
predictores_with_yield.to_csv("../data/preprocessed/{}/cy_bench_8daybins_{}_{}_v7.csv".format(country, crop, country), index=False)
predictores_16day_bins_with_yield.to_csv("../data/preprocessed/{}/cy_bench_16daybins_{}_{}_v7.csv".format(country, crop, country), index=False)